# **DAILY MACRO RELATIVE VALUE TRADE FRAMEWORK**
### Author:  Deborah Akintoye
#### **Purpose:** Convert one macro headline into a backtestable relative-value trade with clear horizon, factor sensitivities, execution logic, and weekly performance tracking.

---


## 1. Trade Metadata & Idea Structuring

**Date:    13-Jan-2026**

**News Source / Catalyst:**

**Markets (s) Impacted:** FX, FI

**Trade Taxonomy:** RV Macro

**Trade Idea (1 sentence):**
> "Recent UK retail data for December, which fell below inflation rates, highlights the pricing pressure on consumers and creates a long EUR/GBP trade opportunity driven by persistent stagflation in the UK.

**Client/Desk Pitch (30 seconds):**
> From recent data, the UK retail slum is unique to the UK's specific "cost of living" crisis and the tax speculations for 2026. Shorting the GBP against the EUR reflects the view that the UK economy is the "sick man of Europe" relative to the Eurozone, who are experiencing growth promoted by lower interest rates.

**Why Now:**
- December's BRC figures prove the *Golden Quarter* for retail was a failure, providing a fundamental catalyst to short the pound before official ONS data on 23-Jan-2026
- GBP/EUR is currently trending near 1.145 (multi-year-lows) with major banks (MUFG) forcasting a further slide towards 1.11
- Markets are currently pricing in at least 2-3 BoE cuts for 2026; entering now capture downward momentum

---

## 2. Macro Hypothesis & Instruments

### **Macro Hypothesis:**
- **Consumption Divergence**: UK real retail growth is negative (1.2% sales vs 3.2% inflation), while Eurozone domestic demand is bolstered by resilient labour markets and real wage growth
- **Yield Compression**: The BoE is pressured to cut rates faster to combat low growth, where the ECB is expected to hold rates steady near 2% as inflation stabilises
- **Fiscal Drag**: UK consumer confidence is suppressed by budgetary uncertainty, contrasted by more neutral or supportive fiscal stances in major Eurozone economies like Germany

**Long Leg (Tickers + Rationale):**
- **FXE** (Invesco CurrencyShares Euro Trust), as this ETF holds physical euros

**Short Leg (Tickers + Rationale):**
- **FXB** (Invesco CurrencyShares British Pound Sterling Trust), as this ETF tracks the price of the pound

**Weights:** FX-Exposure driven weights

---

## 3. Factor Sensitivity & Horizon Alignment

| Factor                | Strength (S/M/W) | Notes |
|----------------------|------------------|-------|
| USD / FX             |        S          |       |
| Oil Beta             |        W          |       |
| Rates Duration       |        S          |       |
| Growth vs Value      |        S          |       |
| Risk-On/Off          |        M          |       |
| China/EM Beta        |        W          |       |
| Volatility Regime    |        M          |       |

**Catalyst Type:** Policy / Data / Regime shift

**Realisation Window:** 2 - 4 months (with our first catalyst being ONS retail data on 23-Jan-2026, and our second catalyst being the BoE's next monetary meeting)

**Holding Period Selected:**  3-6 months ie 60 - 120 trading days (staying power is needed to rigde out the noise)

**Alignment Score:** Good

---


## 4. Risk Controls & Stop Logic

**Position Sizing:** ~97 units (10,000 / 103) for long leg, ~77 units (10,000 / 129) for short leg

**Stop-Loss:** (portfolio-level %)  -0.015 (-1.5%)

**Take-Profit (optional):**  0.03 (3%)

**Re-entry:** No  

**FX-Hedging Required:** No. Since both ETFs are priced in USD,  we are technically short dollars on the long end and long dollars on the short leg. As they cancel each other out our net exposure is purely eur vs GBP

**Liquidity Notes:** Both funds are highly liquid, and have sufficinet AUM for retail and mid-tier institutional size. Execute during London/New York overlap (13:00 - 16:00 GMT) to ensure maximum liquidity and the tightest spreads.

**Failure Mode (tick):**
- [ ] Time
- [ ] Price
- [X] Information

---


# **PYTHON IMPLEMENTATION**

In [108]:
# --- Python Imports ---

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.float_format', lambda x: f'{x:.4f}')


In [109]:
# --- Parameters ---

long_tickers = ["FXE"]    
short_tickers = ["FXB"]                    

start_date = "2020-01-01"                         # Backtest window
holding_period = 60                               # days
stop_loss = -0.015                                 # -2% stop
take_profit = 0.03                                # optional e.g. +0.03 for +3%
long_positioning = 10000/103
short_positioning = 10000/129

# --- Weightings ---

weights_long = long_positioning / (long_positioning + short_positioning)
weights_short = short_positioning / (long_positioning + short_positioning)


In [110]:
# --- Data Download ---

tickers = long_tickers + short_tickers
data = yf.download(tickers, start=start_date)["Close"]

# Drop empty cols & make sure no NA

data = data.dropna(how='all').ffill()

returns = data.pct_change().dropna()


/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_47743/911793947.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date)["Close"]
[*********************100%***********************]  2 of 2 completed


In [111]:
# --- Spread Returns ---

long_returns = returns[long_tickers].mean(axis=1)
short_returns = returns[short_tickers].mean(axis=1)

strategy_returns = long_returns - short_returns


In [112]:
# --- Backtest Simulation ---

trade_pnl = []
trade_dates = []

dates = strategy_returns.index
i = 0
n = len(strategy_returns)

while i < n - holding_period:
    pnl = 0.0
    stopped = False
    entry_date = dates[i]
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
        
        if take_profit is not None and pnl >= take_profit:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
    
    if not stopped:
        trade_pnl.append(pnl)
        trade_dates.append(entry_date)
    
    i += holding_period

trade_results = pd.DataFrame({
    "Trade Date": trade_dates,
    "PnL": trade_pnl
})

trade_results.tail()


,Trade Date,PnL
20,2024-10-10,-0.0174
21,2025-01-07,0.0088
22,2025-04-04,0.0324
23,2025-07-02,0.0142
24,2025-09-26,-0.0020


In [113]:
# --- Performance Summary ---

summary = {
    "Total Trades": len(trade_results),
    "Win Rate": (trade_results["PnL"] > 0).mean(),
    "Average PnL": trade_results["PnL"].mean(),
    "Best Trade": trade_results["PnL"].max(),
    "Worst Trade": trade_results["PnL"].min(),
    "Stop-Out Frequency": (trade_results["PnL"] <= stop_loss).mean()
}

pd.Series(summary)


Total Trades         25.0000
Win Rate              0.2400
Average PnL          -0.0068
Best Trade            0.0324
Worst Trade          -0.0236
Stop-Out Frequency    0.6000
dtype: float64

## Interpretation & Trader Notes

**Absolute Metrics:**
- Total Trades         25.0000
- Win Rate              0.2400
- Average PnL          -0.0068
- Best Trade            0.0324
- Worst Trade          -0.0236
- Stop-Out Frequency    0.6000

**What went right:**
- 

**What went wrong:**
- 

**Factor that actually drove results:**
- 

**Would I trade this live?** (Yes / No and why)
>

---

# **BLOOMBERG-STYLE TRADE BLOTTER**

In [114]:
import uuid
from datetime import datetime

def generate_trade_id(prefix="RV"):
    return f"{prefix}-{uuid.uuid4().hex[:8].upper()}"


In [115]:
# --- Computing Synthetic Basket Price Series ---

basket_long = (data[long_tickers] * weights_long).sum(axis=1)
basket_short = (data[short_tickers] * weights_short).sum(axis=1)

basket_spread = basket_long - basket_short


In [116]:
# --- Backtesting Loop Modified for Blotter ---

blotter_rows = []

i = 0
n = len(strategy_returns)
dates = strategy_returns.index

while i < n - holding_period:
    entry_date = dates[i]
    exit_date = dates[min(i + holding_period - 1, n - 1)]
    
    entry_price = basket_spread.loc[entry_date]
    
    pnl = 0.0
    stopped = False
    trade_status = "CLOSED"
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            exit_date = dates[i + j]
            trade_status = "STOPPED"
            stopped = True
            break
    
    if not stopped and take_profit is not None and pnl >= take_profit:
        exit_date = dates[i + j]
        trade_status = "TAKE_PROFIT"
    
    exit_price = basket_spread.loc[exit_date]
    holding_days = (exit_date - entry_date).days
    
    row = {
        "TradeID": generate_trade_id(prefix="RV"),
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Strategy": "Macro RV Spread",
        "Taxonomy": trade_taxonomy if 'trade_taxonomy' in globals() else "RV Macro Spread",
        "LongLeg": ", ".join(long_tickers),
        "ShortLeg": ", ".join(short_tickers),
        "EntryDate": entry_date.strftime("%Y-%m-%d"),
        "ExitDate": exit_date.strftime("%Y-%m-%d"),
        "EntryPrice": entry_price,
        "ExitPrice": exit_price,
        "GrossPnL(%)": pnl * 100,
        "StopLoss(%)": stop_loss * 100,
        "TakeProfit(%)": take_profit * 100 if take_profit else None,
        "Status": trade_status,
        "HoldingPeriod": holding_days,
        "Notes": "",  # Optional notes field
    }
    blotter_rows.append(row)
    
    i += holding_period

blotter = pd.DataFrame(blotter_rows)
blotter.tail()


,TradeID,Timestamp,Strategy,Taxonomy,LongLeg,ShortLeg,EntryDate,ExitDate,EntryPrice,ExitPrice,GrossPnL(%),StopLoss(%),TakeProfit(%),Status,HoldingPeriod,Notes
20,RV-BCFDF79B,2026-01-13 07:32:27,Macro RV Spread,RV Macro Spread,FXE,FXB,2024-10-10,2024-12-10,1.3177,0.3216,-1.7420,-1.5000,3.0000,STOPPED,61,
21,RV-352B87DC,2026-01-13 07:32:27,Macro RV Spread,RV Macro Spread,FXE,FXB,2025-01-07,2025-04-03,0.5691,1.1895,0.8832,-1.5000,3.0000,CLOSED,86,
22,RV-85777FA2,2026-01-13 07:32:27,Macro RV Spread,RV Macro Spread,FXE,FXB,2025-04-04,2025-07-01,1.7714,2.2574,1.6696,-1.5000,3.0000,CLOSED,88,
23,RV-71C4DE3D,2026-01-13 07:32:27,Macro RV Spread,RV Macro Spread,FXE,FXB,2025-07-02,2025-09-25,2.7712,3.0443,1.4171,-1.5000,3.0000,CLOSED,85,
24,RV-933FCC41,2026-01-13 07:32:27,Macro RV Spread,RV Macro Spread,FXE,FXB,2025-09-26,2025-12-19,3.0093,2.9678,-0.1986,-1.5000,3.0000,CLOSED,84,


In [117]:
blotter.to_csv("trade_blotter.csv", index=False)